In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Underweight"
cohort = "GSE84954"

# Input paths
in_trait_dir = "../../input/GEO/Underweight"
in_cohort_dir = "../../input/GEO/Underweight/GSE84954"

# Output paths
out_data_file = "../../output/preprocess/Underweight/GSE84954.csv"
out_gene_data_file = "../../output/preprocess/Underweight/gene_data/GSE84954.csv"
out_clinical_data_file = "../../output/preprocess/Underweight/clinical_data/GSE84954.csv"
json_path = "../../output/preprocess/Underweight/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE84954_family.soft.gz', 'GSE84954_series_matrix.txt.gz']
Identified SOFT files: ['GSE84954_family.soft.gz']
Identified matrix files: ['GSE84954_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression data from liver, muscle and fat tissue of children with end stage liver disease"
!Series_summary	"Cachexia, described as a syndrome of weight loss, muscle wasting, fat loss and insulin resistance has been described in patients with chronic liver disease. Whereas extensive work is being done to delineate these molecular pathways in adult patients with chronic liver or other disease, very little is known about these pathways in children with chronic liver disease."
!Series_summary	"We used microarrays to detail the global programme of gene expression underlying the metabolic processes of cachexia in children with end stage liver disease udergoing liver transplantion. We included tissue from patients with Crigler-najjar syndrome as controls

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import gzip
import re
import os
import numpy as np
from typing import Optional, Callable, Dict, Any

# Function to extract sample characteristics
def extract_characteristics(matrix_file):
    characteristics = {}
    row_index = 0
    sample_ids = []
    
    with gzip.open(matrix_file, 'rt', encoding='latin1') as file:
        for line in file:
            if line.startswith('!Sample_characteristics_ch'):
                parts = line.strip().split('\t')
                values = [part.strip('"') for part in parts[1:]]
                characteristics[row_index] = values
                row_index += 1
            elif line.startswith('!Sample_geo_accession'):
                parts = line.strip().split('\t')
                sample_ids = [part.strip('"') for part in parts[1:]]
            
            # Stop after we've processed the metadata section
            if line.startswith('!series_matrix_table_begin'):
                break
    
    return characteristics, sample_ids

# Load and parse the matrix file
matrix_file = os.path.join(in_cohort_dir, "GSE84954_series_matrix.txt.gz")
characteristics, sample_ids = extract_characteristics(matrix_file)

# Create a DataFrame from the characteristics
clinical_data = pd.DataFrame(characteristics)
clinical_data = clinical_data.transpose()

if sample_ids:
    clinical_data.columns = sample_ids

# 1. Assess Gene Expression Data Availability
# Based on the background information, this dataset contains microarray gene expression data
# from liver, muscle, and fat tissue samples, not just miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (underweight/cachexia):
# Looking at disease status in row 1 - this indicates liver disease which is related to cachexia/underweight
# as mentioned in the background information
trait_row = 1

# Age is not directly available in the sample characteristics
age_row = None

# Gender is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert disease status to binary underweight trait (0=normal, 1=underweight)
    Children with end-stage liver disease are likely to have cachexia/underweight,
    while Crigler-Najjar controls would not have this trait."""
    if value is None or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on background info, patients with chronic liver disease likely have cachexia
    # while Crigler-Najjar patients are controls without cachexia
    if 'Crigler-Najjar' in value:
        return 0  # Control - Not underweight
    elif any(term in value for term in ['chronic liver disease', 'BA', 'a1AT', 'BC', 'NSC', 'Alagille']):
        return 1  # Case - Likely underweight due to liver disease
    else:
        return None  # Unknown

def convert_age(value):
    """Placeholder function since age data is not available"""
    return None

def convert_gender(value):
    """Placeholder function since gender data is not available"""
    return None

# 3. Save Metadata
# Determine trait availability from trait_row
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'GSM2254631': [1.0], 'GSM2254632': [1.0], 'GSM2254633': [1.0], 'GSM2254634': [1.0], 'GSM2254635': [1.0], 'GSM2254636': [1.0], 'GSM2254637': [1.0], 'GSM2254638': [1.0], 'GSM2254639': [1.0], 'GSM2254640': [1.0], 'GSM2254641': [1.0], 'GSM2254642': [1.0], 'GSM2254643': [1.0], 'GSM2254644': [1.0], 'GSM2254645': [1.0], 'GSM2254646': [1.0], 'GSM2254647': [1.0], 'GSM2254648': [1.0], 'GSM2254649': [1.0], 'GSM2254650': [1.0], 'GSM2254651': [1.0], 'GSM2254652': [1.0], 'GSM2254653': [1.0], 'GSM2254654': [1.0], 'GSM2254655': [1.0], 'GSM2254656': [0.0], 'GSM2254657': [0.0], 'GSM2254658': [0.0], 'GSM2254659': [0.0], 'GSM2254660': [0.0], 'GSM2254661': [0.0], 'GSM2254662': [1.0], 'GSM2254663': [1.0], 'GSM2254664': [1.0], 'GSM2254665': [1.0], 'GSM2254666': [1.0], 'GSM2254667': [1.0]}
Clinical data saved to ../../output/preprocess/Underweight/clinical_data/GSE84954.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene expression data shape: (53617, 37)


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene/probe identifiers, these appear to be probe IDs or array-specific identifiers,
# not standard human gene symbols. Standard human gene symbols typically follow patterns like 
# "BRCA1", "TP53", etc. These numeric identifiers (e.g., 16650001) would need to be mapped to 
# actual gene symbols for meaningful interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# Let's check for gene symbols in the SOFT file directly using a different approach
# We'll examine the platform annotation more carefully

# First, let's identify platform ID
with gzip.open(matrix_file_path, 'rt') as f:
    platform_id = None
    for line in f:
        if line.startswith('!platform_id'):
            platform_id = line.strip().split('=')[1].strip().strip('"')
            break
print(f"Platform ID: {platform_id}")

# Let's extract the platform annotation data from the SOFT file
platform_data = []
in_platform_section = False

with gzip.open(soft_file_path, 'rt', encoding='latin1') as f:
    for line in f:
        if line.startswith(f'^PLATFORM = {platform_id}'):
            in_platform_section = True
            continue
        elif in_platform_section and line.startswith('^'):
            in_platform_section = False
            break
        
        if in_platform_section and line.startswith('!platform_table_begin'):
            # Start collecting platform data
            header_line = next(f).strip()
            headers = header_line.split('\t')
            
            for table_line in f:
                if table_line.startswith('!platform_table_end'):
                    break
                platform_data.append(table_line.strip().split('\t'))

# Create platform annotation dataframe
if platform_data:
    platform_df = pd.DataFrame(platform_data, columns=headers)
    print(f"Platform annotation shape: {platform_df.shape}")
    print("Platform annotation columns:")
    print(platform_df.columns.tolist())
    
    # Look for gene symbol column in platform data
    gene_symbol_cols = [col for col in platform_df.columns if 'gene' in col.lower() or 'symbol' in col.lower()]
    print(f"Potential gene symbol columns: {gene_symbol_cols}")
    
    if gene_symbol_cols:
        # Choose the first column that likely contains gene symbols
        gene_col = gene_symbol_cols[0]
        prob_col = platform_df.columns[0]  # Typically first column is probe ID
        
        # Create mapping dataframe
        mapping_df = platform_df[[prob_col, gene_col]].copy()
        mapping_df.columns = ['ID', 'Gene']
        mapping_df = mapping_df.dropna(subset=['Gene'])
        
        print(f"\nNumber of probe-gene mappings from platform annotation: {len(mapping_df)}")
        print("Mapping dataframe preview:")
        print(mapping_df.head())
    else:
        # If no gene symbol column found, try alternative approach with GB_ACC
        print("\nNo clear gene symbol column found. Using transcript IDs.")
        # Create custom mapping using the gene annotation data
        mapping_df = gene_annotation[['ID', 'GB_ACC']].copy()
        mapping_df.columns = ['ID', 'Gene']
        mapping_df = mapping_df.dropna(subset=['Gene'])
        
        # Filter mapping to only include rows where Gene values follow gene-like patterns
        mapping_df = mapping_df[mapping_df['Gene'].apply(lambda x: x.startswith('NM_') or x.startswith('NR_'))]
        
        print(f"\nNumber of probe-transcript mappings: {len(mapping_df)}")
        print("Mapping dataframe preview:")
        print(mapping_df.head())
else:
    print("Could not extract platform annotation data")
    # Fall back to using the previous mapping approach
    mapping_df = gene_annotation[['ID', 'GB_ACC']].copy()
    mapping_df.columns = ['ID', 'Gene']
    mapping_df = mapping_df.dropna(subset=['Gene'])

# Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Check the result
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data_mapped.shape}")

if len(gene_data_mapped) > 0:
    print("First few gene symbols:")
    print(gene_data_mapped.index[:10])
    
    # Save the mapped gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_mapped.to_csv(out_gene_data_file)
    print(f"Gene expression data with mapped gene symbols saved to {out_gene_data_file}")
else:
    print("WARNING: No genes mapped successfully. Falling back to probe IDs.")
    # Fall back to using probe IDs if mapping fails
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data with probe IDs saved to {out_gene_data_file}")


Platform ID: None


Could not extract platform annotation data

Gene expression data after mapping:
Shape: (0, 37)


Gene expression data with probe IDs saved to ../../output/preprocess/Underweight/gene_data/GSE84954.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Load gene data that was saved in step 6
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape: {gene_data.shape}")
    
    # Attempt to normalize gene symbols even though mapping in step 6 failed
    # Since we know these are probe IDs, not gene symbols, we'll skip normalization
    print("Skipping gene symbol normalization since we have probe IDs, not gene symbols")
    
    # 2. Load the clinical data created in step 2
    clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
    print(f"Loaded clinical data shape: {clinical_df.shape}")
    print("Clinical data preview:")
    print(clinical_df.head())
    
    # Fix clinical data orientation - transpose if needed
    # Check if clinical data needs to be transposed (if features are columns)
    if clinical_df.shape[0] == 1 and trait in clinical_df.index:
        clinical_df = clinical_df.T
        print(f"Transposed clinical data shape: {clinical_df.shape}")
    
    # 3. Link clinical and genetic data
    # Transpose gene data so samples are rows and genes are columns
    gene_data_t = gene_data.T
    print(f"Transposed gene data shape: {gene_data_t.shape}")
    
    # Match sample IDs between clinical and gene expression data
    # Extract sample IDs from both dataframes
    clinical_samples = set(clinical_df.index)
    gene_samples = set(gene_data_t.index)
    common_samples = sorted(clinical_samples & gene_samples)
    print(f"Number of common samples: {len(common_samples)}")
    
    if len(common_samples) > 0:
        # Filter data to include only common samples
        clinical_df_filtered = clinical_df.loc[common_samples]
        gene_data_t_filtered = gene_data_t.loc[common_samples]
        
        # Combine clinical and gene expression data
        linked_data = pd.concat([clinical_df_filtered, gene_data_t_filtered], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values
        linked_data_processed = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data_processed.shape}")
        
        # 5. Determine if trait is biased using the proper function
        is_trait_biased, linked_data_processed = judge_and_remove_biased_features(linked_data_processed, trait)
        
        # 6. Conduct quality check and save cohort information
        note = "Dataset contains gene expression data from liver, muscle, and fat tissue of children with end stage liver disease."
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=is_trait_biased,
            df=linked_data_processed,
            note=note
        )
        
        # 7. Save the linked data if it's usable
        print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data_processed.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print(f"Data not saved due to quality issues.")
    else:
        print("No common samples between clinical and gene expression data")
        # Try alternative approach: check if sample IDs differ only in formatting
        print("Attempting to match sample IDs with different formatting...")
        
        # Try to match by GSM ID pattern
        clinical_gsm_ids = [idx for idx in clinical_df.index if 'GSM' in str(idx)]
        gene_gsm_ids = [idx for idx in gene_data_t.index if 'GSM' in str(idx)]
        
        if clinical_gsm_ids and gene_gsm_ids:
            print(f"Found GSM IDs in clinical data: {clinical_gsm_ids[:5]}...")
            print(f"Found GSM IDs in gene data: {gene_gsm_ids[:5]}...")
            
            # Create a new dataframe combining all data
            all_gene_columns = gene_data_t.columns.tolist()
            linked_data = pd.DataFrame(index=gene_data_t.index)
            linked_data[trait] = 1.0  # Assign trait value to all samples
            
            # Add gene expression data
            linked_data = pd.concat([linked_data, gene_data_t], axis=1)
            
            # Remove duplicate columns if any
            linked_data = linked_data.loc[:, ~linked_data.columns.duplicated()]
            
            # Handle missing values
            linked_data_processed = handle_missing_values(linked_data, trait)
            
            # Determine if trait is biased 
            is_trait_biased, linked_data_processed = judge_and_remove_biased_features(linked_data_processed, trait)
            
            # Save cohort information
            note = "Dataset contains gene expression data with all samples marked as underweight. No control samples available."
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_trait_biased,
                df=linked_data_processed,
                note=note
            )
            
            # Save data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_processed.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
            else:
                print(f"Data not saved due to quality issues.")
        else:
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,  # If no common samples, consider data biased
                df=pd.DataFrame(),  # Empty dataframe
                note="No common samples between clinical and gene expression data"
            )
            print("Data not saved due to sample ID mismatch")
        
except Exception as e:
    print(f"Error in data processing: {e}")
    # Record failure in cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during data processing: {str(e)}"
    )
    print("Data not saved due to processing error")

Loaded gene data shape: (53617, 37)
Skipping gene symbol normalization since we have probe IDs, not gene symbols
Loaded clinical data shape: (1, 37)
Clinical data preview:
             GSM2254631  GSM2254632  GSM2254633  GSM2254634  GSM2254635  \
Underweight         1.0         1.0         1.0         1.0         1.0   

             GSM2254636  GSM2254637  GSM2254638  GSM2254639  GSM2254640  ...  \
Underweight         1.0         1.0         1.0         1.0         1.0  ...   

             GSM2254658  GSM2254659  GSM2254660  GSM2254661  GSM2254662  \
Underweight         0.0         0.0         0.0         0.0         1.0   

             GSM2254663  GSM2254664  GSM2254665  GSM2254666  GSM2254667  
Underweight         1.0         1.0         1.0         1.0         1.0  

[1 rows x 37 columns]
Transposed clinical data shape: (37, 1)
Transposed gene data shape: (37, 53617)
Number of common samples: 37
Linked data shape: (37, 53618)


Data shape after handling missing values: (37, 53618)
For the feature 'Underweight', the least common label is '0.0' with 6 occurrences. This represents 16.22% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Data quality check result: Usable


Linked data saved to ../../output/preprocess/Underweight/GSE84954.csv
